In [1]:
# ==========================================
# File: project_rdkt/notebooks/02_model_tuning_xgb.ipynb
# ==========================================
# [RDKit 专属] XGBoost 参数调优 (强力抗过拟合版)
# 针对: "严重过拟合" 现象
# 策略: 极限制树深 (Depth), 高分裂门槛 (Gamma), 高叶子权重 (Min Child Weight)

import os
import json
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, r2_score
from scipy.stats import randint, uniform, loguniform

# 1. 环境准备
if 'notebooks' in os.getcwd(): os.chdir('..')
print(f"🚀 启动参数调优 (Anti-Overfit Mode) | 项目: RDKit Only")

# 2. 读取数据
try:
    train = pd.read_csv('data/processed/train.csv')
    target_col = 'Target_Log1o2' if 'Target_Log1o2' in train.columns else 'Target'
    X_train = train.drop(columns=[target_col, 'SMILES_Meta'], errors='ignore')
    y_train = train[target_col]
    print(f"✅ 数据加载成功: {X_train.shape}")
except FileNotFoundError:
    raise FileNotFoundError("❌ 找不到数据，请先运行 01_feature_engineering.ipynb")

# 3. 构建管道
pipe = Pipeline([
    ('yeo', PowerTransformer()),
    ('scaler', StandardScaler()),
    ('model', XGBRegressor(objective='reg:squarederror', n_jobs=-1, random_state=42))
])

# 4. 定义"抗过拟合"搜索空间
param_dist = {
    # --- 限制模型复杂度 ---
    'model__n_estimators': randint(50, 400),        # 树少一点
    'model__max_depth': [2, 3],                     # ★核心★: 极浅的树，只学大规律
    'model__learning_rate': loguniform(0.01, 0.1),  # 学习率适中
    'model__min_child_weight': randint(3, 8),       # ★核心★: 叶子节点至少要有 3-8 个样本，防止孤立点拟合

    # --- 增加分裂难度 ---
    'model__gamma': uniform(0.1, 1.0),              # ★核心★: 分裂所需的最小 Loss 减少量，越大越保守

    # --- 正则化 (惩罚项) ---
    'model__reg_alpha': loguniform(0.1, 10.0),      # L1 Lasso
    'model__reg_lambda': loguniform(1.0, 20.0),     # L2 Ridge (加大力度)

    # --- 随机性 ---
    'model__subsample': uniform(0.5, 0.3),          # 0.5~0.8
    'model__colsample_bytree': uniform(0.5, 0.4),   # 0.5~0.9
}

# 5. 执行搜索
print("   ...正在进行抗过拟合参数搜索...")
# 使用 KFold (Shuffle=True)
cv = KFold(n_splits=5, shuffle=True, random_state=42)

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_dist,
    n_iter=100,       # 搜索 100 次
    scoring='neg_mean_squared_error', # 关注误差
    n_jobs=-1,
    verbose=1,
    random_state=42
)

search.fit(X_train, y_train)

# 6. 结果与保存
best_r2 = r2_score(y_train, search.predict(X_train)) # 简单的训练集回测
print("\n" + "="*40)
print(f"🏆 最佳参数找到!")
print(f"   Best CV MSE: {-search.best_score_:.4f}")
print("-" * 40)

os.makedirs('results/models', exist_ok=True)
save_path = 'results/models/best_params_xgb.json'

# 清洗参数键名
cleaned_params = {k.replace('model__', ''): v for k, v in search.best_params_.items()}

with open(save_path, 'w') as f:
    json.dump(cleaned_params, f, indent=4)

print(f"💾 参数已保存: {save_path}")
print("   (请运行 03 号文件进行验证，观察 Train 和 Test 的差距是否缩小)")

🚀 启动参数调优 (Anti-Overfit Mode) | 项目: RDKit Only
✅ 数据加载成功: (105, 126)
   ...正在进行抗过拟合参数搜索...
Fitting 5 folds for each of 100 candidates, totalling 500 fits



🏆 最佳参数找到!
   Best CV MSE: 1.3208
----------------------------------------
💾 参数已保存: results/models/best_params_xgb.json
   (请运行 03 号文件进行验证，观察 Train 和 Test 的差距是否缩小)
